## This script allows you to copy a StoryMap from AGOL to Portal 
#### (or Portal to AGOL, AGOL to AGOL, portal to portal)

1. Get your ID of your original StoryMap Handy
2. Make sure the StoryMap is published - it can be published privately, but it can't be unpublished
3. Have your credentials for AGOL and portal for your orgs handy

More info [here](https://developers.arcgis.com/python/samples/clone-storymap-version2/)

In [3]:
# import libraries

import os
import uuid
import json
import shutil
import tempfile

from arcgis.gis import GIS
from arcgis import __version__

In [ ]:
# Assign a variable to store appropriate version values to differentiate beween each story model.

_version = [int(i) for i in __version__.split('.')]


In [ ]:
# define a function to export the supporting resources to a zip file for the ArcGIS StoryMap model

def export_resources(item, save_path=None, file_name=None):
    """Export's the data's resources as a zip file"""
    url = \
        f'{item._gis._portal.resturl}content/users/{item._user_id}/items/{item.itemid}/resources/export'
    if save_path is None:
        save_path = tempfile.gettempdir()
    if file_name is None:
        file_name = f"{uuid.uuid4().hex[:6]}.zip"
    params = {'f' : 'zip'}
    con = item._gis._portal.con
    resources = con.get(url, params=params,
                        out_folder=save_path,
                        file_name=file_name,
                        try_json=False)
    return resources

In [ ]:
# Connect to the source and destination GIS organizations
# Can change these around but for example we have AGOL to Portal

# Create AGOL connection
agol_user_input = input("AGOL Username: ") # Populate with a username to perform operation
agol = GIS(url='https://agol_name.maps.arcgis.com/', username=agol_user_input) # change agol_name

# Create Portal connection 
portal_user_input = input("Portal Username: ") # Populate with a username to perform operation
portal = GIS(url='https://portal_name.org/portal', username=portal_user_input) #change portal_name

gis = agol # Where the StoryMap presently resides
dest_gis = portal # Where you would like a copy of the StoryMap

In [ ]:
story_map_id = "put_story_map_id_here" # put the item ID for the StoryMap in the quotes (Ex: 358b83b5f776402fa726cfa316aa197c)

story_map = gis.content.get(story_map_id)
if _version <= [1,8,2]:
    resource = export_resources(item=story_map)
else:
    resource = story_map.resources.export()

In [ ]:
# Visualize the Story Map item details to make sure you got the correct StoryMap
story_map

In [ ]:
# Use this to examine the resources used by the Story Map
resource 

In [ ]:
# Collect the Web Maps and Express Maps using the StoryMap's data. Use the set 
# operator each item is collected only once for cloning.
story_map_json = story_map.get_data(try_json=True) # if this chunk isn't working remember to publish your StoryMap!

web_maps = set([v['data']['itemId'] for k, v in story_map_json['resources'].items() \
                if v['type'].lower().find('webmap')>-1])
express_maps = set([v['data']['itemId'] for k, v in story_map_json['resources'].items() \
                    if v['type'].lower().find('expressmap')>-1])

In [ ]:
# Clone each Web Map from the StoryMap and assign a dictionary with the source Web Map id as the key, and the cloned Web Map id as the value
webmap_mapper = {}
for wm in web_maps:
    webmap_to_copy = gis.content.get(wm)
    cloned_webmaps = dest_gis.content.clone_items([webmap_to_copy]) # Clones the WebMap
    webmap_mapper[webmap_to_copy.id] = [i for i in cloned_webmaps if i.type == 'Web Map'][0].id

In [ ]:
# Remap the OLD ItemId to the New Item ID

story_map_text = json.dumps(story_map_json)

for k, v in webmap_mapper.items():
    story_map_text = story_map_text.replace(k, v) # replace the IDs

In [ ]:
# Create a new StoryMap item in the Destination GIS
new_item = dest_gis.content.add({'type' : story_map.type,
                                 'tags' : story_map.tags,
                                 'title' : story_map.title,
                                 'description' : story_map.description,
                                 'typeKeywords' : story_map.typeKeywords,
                                 'extent' : story_map.extent,
                                 'text' :story_map_text}
                                )

In [ ]:
# view new item to make sure it worked
new_item

In [ ]:
# download the original item thumbnail to use to update our new StoryMap
orig_thumbnail = story_map.download_thumbnail(r"your/file/path")
orig_thumbnail = story_map.download_thumbnail(r"C:/Job/sftriage/thumbnails/")

In [ ]:
# make sure this is TRUE
new_item.update(thumbnail=orig_thumbnail)

# check to see the update
new_item


In [ ]:
# get new item id
new_item.id

# THIS IS IMPORTANT FOR EDITING AND VIEWING

#Update the original StoryMap url --- note this is for AGOL to portal 
new_item.update({'url': story_map.url.replace(story_map.url, "https://org_name.org/portal/apps/storymaps/stories/new_item_id")})

# AGOL org to different AGOL org or portal org to portal org
# new_item.update({'url': story_map.url.replace(story_map.id, new_item.id)})

# portal to agol
# new_item.update({'url': story_map.url.replace(story_map.url, "https://storymaps.arcgis.com/stories/new_item_id")})



In [ ]:
# Do this to enable editing - if you can't edit you didn't run this!

with tempfile.NamedTemporaryFile(mode='w', suffix='.json', 
                                 dir=tempfile.gettempdir(), 
                                 delete=False) as jsonfile:
    jsonfile.write(json.dumps(new_item.get_data()))
    new_item.resources.add(file=jsonfile.name)
    type_keywords = [tk for tk in new_item.typeKeywords if 'smdraftresourceid:' not in tk]
    type_keywords.append(f'smdraftresourceid:{os.path.basename(jsonfile.name)}')
    new_item.update({'typeKeywords' : type_keywords})

In [ ]:
# optional - transfer express maps

if len(express_maps) > 0:
    with tempfile.TemporaryDirectory() as d:
        shutil.unpack_archive(filename=resource, extract_dir=d)
        for expmap in express_maps:
            express_draft = os.path.join(d, "draft_"+ expmap)
            express_pub = os.path.join(d, "pub_" + expmap)
            if os.path.isfile(express_pub):
                shutil.copy(express_pub, express_draft)
                new_item.resources.add(express_draft)

In [ ]:
# Check it out-- if it doesn't work double check above step was configured properly
print("your new item can be found here: " + new_item.homepage)


## That's it!